In [10]:
import pandas as pd
import numpy as np

from sklearn.datasets import load_wine
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split

### Датасет "Wine recognition"

In [11]:
wine_dataset = load_wine()
data = wine_dataset['data']
target = wine_dataset['target']

In [12]:
print(wine_dataset['DESCR'])

.. _wine_dataset:

Wine recognition dataset
------------------------

**Data Set Characteristics:**

    :Number of Instances: 178
    :Number of Attributes: 13 numeric, predictive attributes and the class
    :Attribute Information:
 		- Alcohol
 		- Malic acid
 		- Ash
		- Alcalinity of ash  
 		- Magnesium
		- Total phenols
 		- Flavanoids
 		- Nonflavanoid phenols
 		- Proanthocyanins
		- Color intensity
 		- Hue
 		- OD280/OD315 of diluted wines
 		- Proline

    - class:
            - class_0
            - class_1
            - class_2
		
    :Summary Statistics:
    
    ============================= ==== ===== ======= =====
                                   Min   Max   Mean     SD
    ============================= ==== ===== ======= =====
    Alcohol:                      11.0  14.8    13.0   0.8
    Malic Acid:                   0.74  5.80    2.34  1.12
    Ash:                          1.36  3.23    2.36  0.27
    Alcalinity of Ash:            10.6  30.0    19.5   3.3
    Ma

---

In [13]:
X_train_wine, X_test_wine, y_train_wine, y_test_wine = train_test_split(data, target, random_state=2, test_size=0.33)

In [14]:
tree = DecisionTreeClassifier(max_features=X_train_wine.shape[1], random_state=1)
tree.fit(X_train_wine, y_train_wine)
(y_test_wine == tree.predict(X_test_wine)).mean()

0.864406779661017

**Статья на вики про [случайный лес](https://ru.wikipedia.org/wiki/Random_forest).**

Случайный лес:

* Обучить много глубоких деревьев для решения задачи;
* Усреднить их ответы на этапе предсказания.

Задание:

    1) Реализуйте случайный лес решающих деревьев пользуясь классом DecisionTreeClassifier в качестве базового алгоритма;
    2) Обучите ваш классификатор с на данных Wine recognition dataset;
    3) Сравните полученный результат с бейзлайном на одном Decision tree;
    4) Постройте график зависимости точности от количества деревьев;
    5) Постройте график зависимости точности от параметра max_features в базовых деревьях;
    6) Постройте шрафик зависимости точности от глубины базовых алгоритмов.
    
Обратите внимание, что количество классов может быть > 2.

In [35]:
class MyDecisionTreeClassifier:

    def __init__(self, max_depth=None, max_features=None, min_leaf_samples=None, max_classes=None):
        self.max_depth = max_depth
        self.max_features = max_features
        self.min_leaf_samples = min_leaf_samples
        self._node = {
            'left': None,
            'right': None,
            'feature': None,
            'threshold': None,
            'depth': 0,
            'classes_proba': None
        }
        self.tree = None  # словарь в котором будет храниться построенное дерево
        self.classes = max_classes  # список меток классов

    def fit(self, X, y):
        if self.classes is None:
            self.classes = np.unique(y)
        
        self.tree = {'root': self._node.copy()}  # создаём первую узел в дереве
        self._build_tree(self.tree['root'], X, y)  # запускаем рекурсивную функцию для построения дерева
        return self

    def predict_proba(self, X):
        proba_preds = []
        for x in X:
            preds_for_x = self._get_predict(self.tree['root'], x)  # рекурсивно ищем лист в дереве соответствующий объекту
            proba_preds.append(preds_for_x)
        return np.array(proba_preds)

    def predict(self, X):
        proba_preds = self.predict_proba(X)
        preds = proba_preds.argmax(axis=1).reshape(-1, 1)
        return preds

    def get_best_split(self, X, y):
        j_best, t_best = None, None
        Q_best = -1
        
        if self.max_features:
            features = np.random.choice(X.shape[1], size=self.max_features, replace=False)
        else:
            features = list(range(X.shape[1]))

        for i in features:
            sorted = X[X[:, i].argsort()]
            y_sorted = y[X[:, i].argsort()]
            for t in range(1, len(y)):
                if sorted[t, i] == sorted[t - 1, i]:
                    continue
                y_left = y_sorted[:t]
                y_right = y_sorted[t:]
                Q = self.calc_Q(y_sorted, y_left, y_right)
                if Q > Q_best:
                    Q_best = Q
                    j_best = i
                    t_best = (sorted[t, i] + sorted[t - 1, i]) / 2

        return j_best, t_best, X[:, j_best] <= t_best, X[:, j_best] > t_best

    def calc_Q(self, y, y_left, y_right):
        return self.gini(y) - (len(y_left) / len(y) * self.gini(y_left) + len(y_right) / len(y) * self.gini(y_right))

    def gini(self, y):
        probs = np.bincount(y, minlength=len(self.classes))
        probs = probs / np.sum(probs)
        return 1 - np.sum(probs ** 2)

    def _build_tree(self, curr_node, X, y):

        if curr_node['depth'] == self.max_depth:  # выход из рекурсии если построили до максимальной глубины
            curr_node['classes_proba'] = {c: (y == c).mean() for c in self.classes}  # сохраняем предсказания листьев дерева перед выходом из рекурсии
            return

        if len(np.unique(y)) == 1:  # выход из рекурсии значения если "y" одинковы для все объектов
            curr_node['classes_proba'] = {c: (y == c).mean() for c in self.classes}
            return

        j, t, left_ids, right_ids = self.get_best_split(X, y)  # нахождение лучшего разбиения

        curr_node['feature'] = j  # признак по которому производится разбиение в текущем узле
        curr_node['threshold'] = t  # порог по которому производится разбиение в текущем узле

        left = self._node.copy()  # создаём узел для левого поддерева
        right = self._node.copy()  # создаём узел для правого поддерева

        left['depth'] = curr_node['depth'] + 1  # увеличиваем значение глубины в узлах поддеревьев
        right['depth'] = curr_node['depth'] + 1

        curr_node['left'] = left
        curr_node['right'] = right

        self._build_tree(left, X[left_ids], y[left_ids])  # продолжаем построение дерева
        self._build_tree(right, X[right_ids], y[right_ids])

    def _get_predict(self, node, x):
        if node['threshold'] is None:  # если в узле нет порога, значит это лист, выходим из рекурсии
            return [node['classes_proba'][c] for c in self.classes]

        if x[node['feature']] <= node['threshold']:  # уходим в правое или левое поддерево в зависимости от порога и признака
            return self._get_predict(node['left'], x)
        else:
            return self._get_predict(node['right'], x)

In [48]:
class MyRandomForestClassifier:
    
    def __init__(self, max_features, n_estimators=100, max_depth=None):
        self.n_estimators = n_estimators
        self.max_features = max_features
        self.max_depth = max_depth
        self.forest = []
        
    def fit(self, X, y):
        
        self.forest = []
        self.classes = np.unique(y)
        
        while len(self.forest) != self.n_estimators:
            dt = MyDecisionTreeClassifier(max_depth=self.max_depth, max_features=self.max_features, max_classes=self.classes)
            bootstrap = np.random.choice(len(y), size=len(y), replace=True)
            
            dt.fit(X[bootstrap], y[bootstrap])
            
            self.forest.append(dt)
        
        return self
    
    def predict(self, X):
        preds = self.predict_proba(X)
        return np.argmax(preds, axis=1)
    
    def predict_proba(self, X):
        
        preds = np.zeros(len(self.classes))
        for tree in self.forest:
            preds = preds + tree.predict_proba(X)
            
        preds = preds / self.n_estimators
            
        return preds

In [55]:
rf = MyRandomForestClassifier(max_features=4, n_estimators=100)
rf.fit(X_train_wine, y_train_wine)
(y_test_wine == rf.predict(X_test_wine)).mean()

1.0

### Датасет "Boston house prices"

In [56]:
import pandas as pd
import numpy as np

data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
data_boston = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
target_boston = raw_df.values[1::2, 2]

# data_boston = boston_dataset['data']
# target_boston = boston_dataset['target']
# 
# print(boston_dataset['DESCR'])

In [57]:
X_train_boston, X_test_boston,\
y_train_boston, y_test_boston = train_test_split(data_boston, target_boston, random_state=2, test_size=0.33)

---

**Статья на вики про [градиентный бустинг](https://en.wikipedia.org/wiki/Gradient_boosting).**

**Градиентный бустинг**

* Инициализировать ответ ансамбля $a_0$ нулями;
* Вычислить градиент функционала потерь $L(y - a_0)$;
* Обучить базовый $b_1$ регрессор предсказывать антиградиент $-L(y - a_0)$;
* Добавить базовый алгоритм $b_1$ с весом $\eta$ к композиции;
* Повторять пока ошибка уменьшается на валидации;

Задание:

    1) Реализуйте градиентный бустинг над решающими деревьями пользуясь классом DecisionTreeRegressor в качестве базового алгоритма;
    2) Обучите ваш регрессор на данных Boston house prices;
    3) Сравните полученный результат по метрике MAPE с RandomForestRegressor(n_estimators=2000) из sklearn;
    4) Попробуйте подобрать такие learning_rate и n_estimators для градиентного бустинга, чтобы ошибка MAPE была меньше 10%.

In [168]:
class MyDecisionTreeRegressor:

    def __init__(self, max_depth=None, max_features=None, min_leaf_samples=None):
        self.max_depth = max_depth
        self.max_features = max_features
        self.min_leaf_samples = min_leaf_samples
        self._node = {
            'left': None,
            'right': None,
            'feature': None,
            'threshold': None,
            'depth': 0,
            'mean': None
        }
        self.tree = None  # словарь в котором будет храниться построенное дерево

    def fit(self, X, y):
        self.tree = {'root': self._node.copy()}  # создаём первую узел в дереве
        self._build_tree(self.tree['root'], X, y)  # запускаем рекурсивную функцию для построения дерева
        return self

    def predict(self, X):
        preds = []
        for x in X:
            preds_for_x = self._get_predict(self.tree['root'], x)
            preds.append(preds_for_x)
        return np.array(preds)

    def get_best_split(self, X, y):
        j_best, t_best = None, None
        Q_best = -1

        if self.max_features:
            features = np.random.choice(X.shape[1], size=self.max_features, replace=False)
        else:
            features = list(range(X.shape[1]))

        for i in features:
            sorted = X[X[:, i].argsort()]
            y_sorted = y[X[:, i].argsort()]
            for t in range(1, len(y)):
                if sorted[t, i] == sorted[t - 1, i]:
                    continue
                y_left = y_sorted[:t]
                y_right = y_sorted[t:]
                Q = self.calc_Q(y_sorted, y_left, y_right)
                if Q > Q_best:
                    Q_best = Q
                    j_best = i
                    t_best = (sorted[t, i] + sorted[t - 1, i]) / 2

        return j_best, t_best, X[:, j_best] <= t_best, X[:, j_best] > t_best

    def calc_Q(self, y, y_left, y_right):
        return self.mse(y) - (len(y_left) / len(y) * self.mse(y_left) + len(y_right) / len(y) * self.mse(y_right))

    def mse(self, y):
        return np.mean((y - np.mean(y)) ** 2)

    def _build_tree(self, curr_node, X, y):

        if curr_node['depth'] == self.max_depth:  # выход из рекурсии если построили до максимальной глубины
            curr_node['mean'] = y.mean()  # сохраняем предсказания листьев дерева перед выходом из рекурсии
            return

        if len(np.unique(y)) == 1:  # выход из рекурсии значения если "y" одинковы для все объектов
            curr_node['mean'] = y.mean()
            return

        j, t, left_ids, right_ids = self.get_best_split(X, y)  # нахождение лучшего разбиения

        curr_node['feature'] = j  # признак по которому производится разбиение в текущем узле
        curr_node['threshold'] = t  # порог по которому производится разбиение в текущем узле

        left = self._node.copy()  # создаём узел для левого поддерева
        right = self._node.copy()  # создаём узел для правого поддерева

        left['depth'] = curr_node['depth'] + 1  # увеличиваем значение глубины в узлах поддеревьев
        right['depth'] = curr_node['depth'] + 1

        curr_node['left'] = left
        curr_node['right'] = right

        self._build_tree(left, X[left_ids], y[left_ids])  # продолжаем построение дерева
        self._build_tree(right, X[right_ids], y[right_ids])

    def _get_predict(self, node, x):
        if node['threshold'] is None:  # если в узле нет порога, значит это лист, выходим из рекурсии
            return node['mean']

        if x[node['feature']] <= node['threshold']:  # уходим в правое или левое поддерево в зависимости от порога и признака
            return self._get_predict(node['left'], x)
        else:
            return self._get_predict(node['right'], x)

In [169]:
class MyGradientBoostingRegressor:

    def __init__(self, n_estimators=100, max_depth=2, learing_rate=0.1, max_features=None):
        self.n_estimators = n_estimators
        self.learing_rate = learing_rate
        self.max_depth = max_depth
        self.max_features = max_features
        self.estimators = []

    def fit(self, X, y):
        self.estimators = []
        self.mean = np.mean(y)
        curr_pred = np.zeros_like(y)
        for i in range(self.n_estimators):
            estimator = MyDecisionTreeRegressor(max_depth=self.max_depth, max_features=self.max_features)
            estimator.fit(X, y - curr_pred)
            self.estimators.append(estimator)
            curr_pred += estimator.predict(X) * self.learing_rate

        return self

    def predict(self, X):
        predict = np.zeros(X.shape[0])
        for estimator in self.estimators:
            predict += estimator.predict(X) * self.learing_rate

        return predict

In [172]:
gbm = MyGradientBoostingRegressor(n_estimators=70, learing_rate=0.15, max_depth=3)

gbm.fit(X=X_train_boston, y=y_train_boston)
preds = gbm.predict(X_test_boston)

(abs(y_test_boston - gbm.predict(X_test_boston)) / y_test_boston).mean()

0.1093034656186621

In [165]:
preds

array([1.69724775e+02, 1.69724775e+02, 6.79799243e+06, 1.69724775e+02,
       1.69609942e+02, 1.69609942e+02, 1.69724775e+02, 1.69724775e+02,
       1.69724775e+02, 1.69724775e+02, 1.69724775e+02, 1.69724775e+02,
       1.69724775e+02, 1.69724775e+02, 1.69609942e+02, 1.69724775e+02,
       1.69609942e+02, 1.69609942e+02, 1.69609942e+02, 1.69609942e+02,
       1.69724775e+02, 1.69724775e+02, 6.79786280e+06, 1.69609942e+02,
       1.69609942e+02, 1.69724775e+02, 6.79798994e+06, 6.79786280e+06,
       6.79786280e+06, 1.69724775e+02, 1.69724775e+02, 1.69724775e+02,
       1.69724775e+02, 1.69724775e+02, 1.69609942e+02, 1.69609942e+02,
       1.69609942e+02, 1.69724775e+02, 1.69724775e+02, 1.69724775e+02,
       1.69724775e+02, 1.69609942e+02, 6.79786280e+06, 1.69609942e+02,
       1.69724775e+02, 1.69609942e+02, 6.79798994e+06, 1.69609942e+02,
       1.69724775e+02, 1.69609942e+02, 1.69724775e+02, 6.79785431e+06,
       1.69609942e+02, 1.92588986e+02, 1.69724775e+02, 1.69724775e+02,
      